<a href="https://colab.research.google.com/github/Chocoding1/Machine_Learning_Deep_Learning/blob/main/07_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **심층 신경망**

## 2개의 층

In [1]:
# 케라스 API를 사용하여 패션 MNIST 데이터셋 불러오기
from tensorflow import keras

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [2]:

# 데이터 전처리
from sklearn.model_selection import train_test_split

# 이미지의 픽셀값 0 ~ 1 사이로 변환
train_scaled = train_input / 255.0
# 2차원 배열 -> 1차원 배열
train_scaled = train_scaled.reshape(-1, 28*28)
# 훈련 세트, 검증 세트 분리
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

위의 인공 신경망 모델의 층을 2개 추가해보자. 여기서 만들 모델의 대략적인 구조는 다음 그림과 같다.<br>
<img src = "https://smartstuartkim.files.wordpress.com/2019/02/mlp.png" height = 300 width = 500>

입력층과 출력층 사이에 있는 모든 층을 **은닉층(hidden layer)**라고 한다.<br>

은닉층에는 활성화 함수가 적용되는데, 출력층에서의 활성화 함수가 시그모이드 함수와 소프트맥스 함수로 한정되어 있는 것과 달리 은닉층의 활성화 함수는 비교적 자유롭다. 대표적으로 시그모이드 함수와 렐루(Relu) 함수 등을 사용한다.

note) 은닉층에 활성화 함수를 적용하는 이유<br>
선형적인 계산에 비선형성을 주기 위해서

note) 회귀를 위한 신경망의 출력층에서는 어떤 활성화 함수를 사용하나?<br>
위의 시그모이드와 소프트맥스는 분류를 위한 신경망의 출력층에서 사용한다.(클래스에 대한 확률을 출력하기 위해)<br>
그러나 회귀의 출력은 임의의 어떤 숫자이므로 활성화 함수를 적용할 필요가 없다. 즉 출력층의 선형 방정식의 계산을 그대로 출력한다. 이렇게 하려면 Dense 층의 activation 매개변수에 아무런 값을 지정하지 않으면 된다.

이제 시그모이드 활성화 함수를 사용한 은닉층과 소프트맥스 함수를 사용한 출력층을 만들어보자.

In [3]:
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))
dense2 = keras.layers.Dense(10, activation='softmax')

dense1 : 은닉층 / 100개의 뉴런을 가진 밀집층(은닉층의 뉴런 개수를 정하는 데에는 특별한 기준이 없다. 경험이 필요 / 그러나 한 가지, 적어도 출력층의 뉴런보다는 많게 만들어야 한다.)<br>
dense2 : 출력층 / 10개의 클래스를 분류하므로 10개의 뉴런을 가진 밀집층

## 심층 신경망 만들기

Sequential 클래스의 객체를 만들 때 여러 개의 층을 추가하려면 각 층을 리스트로 만들어 전달해야 한다. 주의할 점은 출력층을 가장 마지막에 둬야 한다. 즉 해당 리스트는 가장 처음 등장하는 은닉층에서 마지막 출력층의 순서로 나열해야 한다.

In [4]:
model = keras.Sequential([dense1, dense2])

keras는 모델의 summary() 메서드를 호출하면 층에 대한 유용한 정보를 얻을 수 있다.

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Output Shape의 첫 번째 자원은 샘플의 개수를 나타내는데, 샘플의 개수가 아직 정의되지 않았기 때문에 None이다. keras 모델의 fit() 메서드에 훈련 데이터를 주입하면 이 데이터를 한 번에 모두 사용하지 않고 미니배치 경사 하강법을 사용한다.<br>
keras의 기본 미니배치 크기는 32이다. 이 값은 fit() 메서드의 batch_size 매개변수로 바꿀 수 있다. 따라서 샘플 개수를 고정하지 않고 어떤 배치 크기에도 유연하게 대응할 수 있도록 None으로 설정한다.<br>

두 번째 자원인 100은 은닉층의 뉴런 개수를 100으로 지정했기 때문에 나온 당연한 결과이다. 즉 샘플마다 784개의 픽셀값이 은닉층을 통과하면서 100개의 특성으로 압축되었다.

마지막으로는 모델 파라미터 개수가 출력된다.<br>
78500 = 785(픽셀 수) x 100(가중치 = 뉴런 수) + 100(절편 = 뉴런 수)

summary() 메서드의 마지막에는 총 모델 파라미터 개수와 훈련되는 파라미터 개수가 동일하게 79,510으로 나온다. 은닉층과 출력층의 파라미터 개수를 합친 값이다. 그 아래 훈련되지 않는 파라미터(Non-trainable params)는 0으로 나오는데, 간혹 경사 하강법으로 훈련되지 않는 파라미터를 가진 층이 있다.

## 층을 추가하는 다른 방법

1. Sequential 클래스 생성자 안에 바로 입력<br>
dense1과 dense2처럼 밀집층 객체를 따로 저장하여 쓸 일이 없기 때문에 Sequential 클래스의 생성자 안에서 바로 Dense 클래스의 객체를 만드는 경우가 많다.<br>
이렇게 작업하면 추가되는 층을 한눈에 쉽게 알아보는 장점이 있다.

In [6]:
model = keras.Sequential([
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),
    keras.layers.Dense(10, activation='softmax', name='output')
], name='패션 MNIST 모델')

위에서 모델의 이름과 층의 이름을 지정했는데, 모델의 이름과 달리 층의 이름은 반드시 영문이어야 한다.

In [7]:
# 모델과 층의 이름 확인
model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


위의 방법이 편리하지만 층이 아주 많아지면 Sequential 클래스 생성자가 매우 길어지고, 또 조건에 따라 층을 추가할 수도 없다.

2. add() 메서드 사용<br>
Dense 클래스의 객체를 따로 변수에 담지 않고 바로 add() 메서드로 전달한다. 이 방법은 한눈에 추가되는 층을 볼 수 있고 프로그램 실행 시 동적으로 층을 선택하여 추가할 수 있다.

In [8]:
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))

In [9]:
# 모델 정보 확인
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               78500     
                                                                 
 dense_3 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
# 모델 훈련
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 8s 5ms/step - loss: 0.5694 - accuracy: 0.8060
Epoch 2/5
1500/1500 [==============================] - 8s 5ms/step - loss: 0.4128 - accuracy: 0.8511
Epoch 3/5
1500/1500 [==============================] - 7s 4ms/step - loss: 0.3779 - accuracy: 0.8637
Epoch 4/5
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3555 - accuracy: 0.8719
Epoch 5/5
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3381 - accuracy: 0.8778


인공 신경망에 몇 개의 층을 추가하더라도 compile() 메서드와 fit() 메서드 사용법은 동일하다. 이것이 케라스 API의 장점

## 렐루 함수

초창기 인공 신경망의 은닉층에 많이 사용된 활성화 함수인 시그모이드 함수는 단점이 있다.<br>
이 함수의 오른쪽과 왼쪽 끝으로 갈수록 그래프가 누워있기 때문에 올바른 출력을 만드는데 신속하게 대응하지 못한다.<br>
<img src = "https://images.velog.io/images/oooops/post/ba81e0e7-b543-4887-ad2f-02c1a053d47c/image.png" height = 300 width = 400><br>
특히 층이 많은 심층 신경망일수록 그 효과가 누적되어 학습을 더 어렵게 만든다.

이를 개선하기 위해 제안된 다른 종류의 활성화 함수가 **렐루(Relu)** 함수이다.<br>
렐루 함수는 간단하다. 입력이 양수일 경우 마치 활성화 함수가 없는 것처럼 그냥 입력을 통과시키고 음수일 경우에는 0으로 만든다.<br>
<img src = "https://mblogthumb-phinf.pstatic.net/MjAyMDAyMjVfOTIg/MDAxNTgyNjA4MzI2NDA5.e0VyX0yrhE5gtfPjni7IxF5kpArCeByreQsdOMB0240g.CWwTi57bPtAK6C7eLmRn1ED2RE8Lm_C6sVIwMGJS1Akg.PNG.handuelly/image.png?type=w800" height = 250 width = 350><br>
렐루 함수는 max(0,z)와 같이 쓸 수 있다. 이 함수는 z가 0보다 크면 z를 출력하고 z가 0보다 작으면 0을 출력한다. 렐루 함수는 특히 이미지 처리에서 좋은 성능을 낸다고 알려져 있다. 렐루 함수는 특히 이미지 처리에서 좋은 성능을 낸다고 알려져 있다.

케라스에서 제공하는 편리한 층 하나를 더 살펴보면,<br>
기존에 패션 MNIST 데이터는 28 x 28 크기이기 때문에 인공 신경망에 주입하기 위해 넘파이 배열의 reshape() 메서드를 사용해 1차원으로 펼쳤다. 직접 이렇게 1차원으로 펼쳐도 좋지만 케라스에서는 이를 위한 Flatten 층을 제공한다.<br>
사실 Flatten 클래스는 배치 차원을 제외하고 나머지 입력 차원을 모두 일렬로 펼치는 역할만 한다. 즉 입력에 곱해지는 가중치나 절편이 없다. 따라서 인공 신경망의 성능을 위해 기여하는 바는 없다. 하지만 Flatten 클래스를 층처럼 입력층과 은닉층 사이에 추가하기 때문에 층이라고 부른다.<br>
Flatten 층은 다음 코드처럼 입력층 바로 뒤에 추가한다.

In [11]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

첫 번째 Dense 층에 있던 input_shape 매개변수를 Flatten 층으로 옮겼다. 또한 첫 번째 Dense 층의 활성화 함수를 'relu'로 바꿨다.<br>
하지만 Flatten 클래스는 학습하는 층이 아니기 때문에 이 신경망을 깊이가 3인 신경망이라고 부르진 않는다.

In [12]:
# summary() 메서드를 통해 이를 직접 확인
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               78500     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


예상대로 Flatten 클래스에 포함된 모델 파라미터는 0개이다.<br>
케라스의 Flatten 층을 신경망 모델에 추가하면 입력값의 차원을 짐작할 수 있는 것이 또 하나의 장점이다.<br>
입력 데이터에 대한 전처리 과정을 가능한 모델에 포함시키는 것이 케라스 API의 철학 중 하나이다.

In [13]:
# 훈련 데이터를 다시 준비해서 모델 훈련(앞의 코드와 동일하지만 reshape() 메서드 사용 x)
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()
train_scaled = train_input / 255.0
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

모델을 컴파일하고 훈련하는 것은 이전과 동일

In [14]:
# 모델 훈련
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5323 - accuracy: 0.8113
Epoch 2/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3934 - accuracy: 0.8575
Epoch 3/5
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3553 - accuracy: 0.8727
Epoch 4/5
1500/1500 [==============================] - 7s 4ms/step - loss: 0.3327 - accuracy: 0.8802
Epoch 5/5
1500/1500 [==============================] - 5s 4ms/step - loss: 0.3190 - accuracy: 0.8838


시그모이드 함수를 사용했을 때와 비교하면 성능이 조금 향샹되었다.

In [15]:
# 검증 세트에서의 성능 확인
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 2s 3ms/step - loss: 0.3382 - accuracy: 0.8801


[0.3381887674331665, 0.8800833225250244]

## 옵티마이저

케라스는 다양한 종류의 경사 하강법 알고리즘을 제공한다. compile() 메서드는 케라스의 기본 경사 하강법 알고리즘은 RMSprop을 사용했다. 이러한 경사 하강법 알고리즘들을 **옵티마이저(optimizer)**라고 부른다. 이 옵티마이저도 사람이 지정해줘야 하는 하이퍼파라미터이며, RMSprop의 학습률 또한 하이퍼파라미터 중 하나이다.

가장 기본적인 옵티마이저는 확률적 경사 하강법인 SGD이다. 이름은 SGD이지만 기본적으로 미니배치를 사용한다.

SGD 옵티마이저를 사용하려면 compile() 메서드의 optimizer 매개변수를 'sgd'로 지정한다.

In [16]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')

다른 입력이지만 동일한 의미의 코드가 있다.<br>
위의 옵티마이저는 tensorflow.keras.optimizer 패키지 아래 SGD 클래스로 구현되어 있다. 'sgd' 문자열은 이 클래스의 기본 설정 매개변수로 생성한 객체와 동일하다.

In [18]:
# 다음의 코드는 위의 코드와 정확히 동일
sgd = keras.optimizers.SGD()
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')

note) sgd와 'sgd'의 차이는?<br>
원래 sgd = keras.optimizers.SGD()처럼 SGD 클래스 객체를 만들어 사용해야 하는데, 번거로움을 피하고자 'sgd'라 지정하면 자동으로 SGD 클래스 객체를 만들어 준다.

SGD 클래스의 학습률을 바꾸고 싶으면 learning_rate 매개변수에 원하는 학습률을 지정하여 사용한다. (SGD learning_rate 기본값 : 0.01)

In [19]:
sgd = keras.optimizers.SGD(learning_rate=0.1)

기본 경사 하강법 옵티마이저는 모두 SGD 클래스에서 제공한다. SGD 클래스의 옵티마이저인 momentum 매개변수의 기본값은 0이다. 이를 0보다 큰 값으로 지정하면 마치 이전의 그레이디언트를 가속도처럼 사용하는 **모멘텀 최적화(momentum optimization)**를 사용한다. 보통 momentum 매개변수는 0.9 이상을 지정한다.

또한 SGD 클래스의 또 다른 옵티마이저는 nesterov 매개변수를 기본값 False에서 True로 바꾸면 **네스테로프 모멘텀 최적화**(또는 **네스테로프 가속 경사**)를 사용한다.

In [20]:
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)

네스테로프 모멘텀은 모멘텀 최적화를 2번 반복하여 구현한다. 대부분의 경우 네스테로프 모멘텀 최적화가 기본 확률적 경사 하강법보다 더 나은 성능을 제공한다. 모델이 최적점에 가까이 갈수록 학습률을 낮출 수 있기 때문이고, 이렇게 하면 안정적으로 최적점에 수렴할 가능성이 높다. 이런 학습률을 **적응적 학습률(adaptive learning rate)**이라고 한다. 이런 방식들을 학습률 매개변수를 튜닝하는 수고를 덜 수 있는 것이 장점이다.

적응적 학습률을 사용하는 대표적인 옵티마이저 : **Adagrad**, **RMSprop**<br>
각각 compile() 메서드의 optimizer 매개변수에 'adagrad'와 'rmsprop'으로 지정할 수 있다. optimizer 매개변수의 기본값이 'rmsprop'<br>
이 두 옵티마이저의 매개변수를 바꾸고 싶다면 SGD와 같이 Adagrad와 RMSprop 클래스 객체를 만들어 사용하면 된다.

In [21]:
# Adagrad 객체 생성
adagrad = keras.optimizers.Adagrad()
model.compile(optimizer=adagrad, loss='sparse_categorical_crossentropy', metrics='accuracy')

In [23]:
# RMSprop 객체 생성
rmsprop = keras.optimizers.RMSprop()
model.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics='accuracy')

모멘텀 최적화와 RMSprop의 장점을 접목한 것이 **Adam**이다. Adam은 RMSprop와 함께 맨처음 시도해 볼 수 있는 좋은 알고리즘이다. Adam 클래스도 keras.optimizers 패키지 아래에 있다. 적응적 학습률을 사용하는 이 3개의 클래스는 learning_rate 매개변수의 기본값으로 모두 0.001을 사용한다.

여기서는 Adam 클래스의 매개변수 기본값을 사용해 패션 MNIST 모델을 훈련해 보자.

In [24]:
# 모델 생성
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [25]:
# compile() 메서드의 optimizer를 'adam'으로 설정하고 5번의 에포크동안 훈련
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 9s 5ms/step - loss: 0.5277 - accuracy: 0.8160
Epoch 2/5
1500/1500 [==============================] - 10s 7ms/step - loss: 0.3921 - accuracy: 0.8585
Epoch 3/5
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3517 - accuracy: 0.8730
Epoch 4/5
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3251 - accuracy: 0.8815
Epoch 5/5
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3041 - accuracy: 0.8875


In [26]:
# 검증 세트에서의 성능 확인
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 2ms/step - loss: 0.3237 - accuracy: 0.8852


[0.3237314522266388, 0.8851666450500488]